In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm

# 
import numpy as np
import os
from tqdm import trange
import parmap
import glob
from sklearn.decomposition import PCA
#import umap

from tqdm import tqdm
import sleap

import sklearn.experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 
from scipy.io import loadmat
import scipy
import scipy.ndimage
# 

#import numba
#from numba import jit
import parmap

# 
from track import track
from convert import convert
from ethogram import ethogram

Autosaving every 180 seconds


In [139]:
####################################################
################# EVALUTE ERRORS ###################
####################################################

def get_matrices(cohort, labels_gt, labels_pr):

    if cohort == 'cohort1':
        tp = {'male':0, 
              'female':0,
              'pup shaved':0,
              'pup unshaved':0
              }

        fp = {'male':0, 
               'female':0,
               'pup shaved':0,
               'pup unshaved':0
              }

        fn = {'male':0, 
               'female':0,
               'pup shaved':0,
               'pup unshaved':0
              }

        n_labeled = {'male':0, 
                       'female':0,
                       'pup shaved':0,
                       'pup unshaved':0
                      }
    else:
        tp = {'male':0, 
          'female':0,
          'pup1':0,
          'pup2':0,
          'pup3':0,
          'pup4':0
          }

        fp = {'male':0, 
          'female':0,
          'pup1':0,
          'pup2':0,
          'pup3':0,
          'pup4':0
          }
        
        fn = {'male':0, 
          'female':0,
          'pup1':0,
          'pup2':0,
          'pup3':0,
          'pup4':0
          }
        
        n_labeled = {'male':0, 
          'female':0,
          'pup1':0,
          'pup2':0,
          'pup3':0,
          'pup4':0
          }
    
    
    # loop over all labeled frames
    for lf_gt, lf_pr in zip(labels_gt, labels_pr):
        assert lf_gt.frame_idx == lf_pr.frame_idx  # weak check but it's what we have until we fix the video metadata serialization
        #n_gt += len(lf_gt)
        #n_pr += len(lf_pr)

        positive_pairs, false_negatives = sleap.nn.evals.match_instances(lf_gt, lf_pr)
        #FN = len(false_negatives)  # gt instances that were not found
        #FP = len(lf_pr) - len(positive_pairs)  # extra pr instances

                
        # 
        for inst_gt, inst_pr, _ in positive_pairs:
           
            # 
            if inst_gt.track is None:  # gt not id labeled
                #gt_without_id += 1
                continue
                
            # 
            if inst_pr.track is None:  # pr not id labeled
                fn[inst_pr.track.name]+=1
                continue

            # 
            if inst_pr.track.name == inst_gt.track.name:
                tp[inst_pr.track.name]+=1

            else:
                fp[inst_pr.track.name]+=1
                pass 
        
        # add animal specific 
        for inst_gt in false_negatives:
            if inst_gt.track is not None:
                fn[inst_gt.track.name]+=1
           
    return tp, fp, fn


########################################################################

import sleap
import os
import pickle
from tqdm import trange

sleap.disable_preallocation()
sleap.versions()

# 
cohort = 'cohort1'
time_of_day = 'night'
root_dir = os.path.join('/media/cat/256GB/dan/',cohort,time_of_day)

#
if cohort=='cohort1':
    tracks = ['male', 'female', 'pup shaved', 'pup unshaved']
    n_animals=4
else:
    tracks = ['male', 'female', 'pup1', 'pup2', 'pup3','pup4']
    n_animals=6

    
# load gt labels
n_samples = [100, 200, 300, 400, 450]  # NEED TO CHANGE THIS TO CORRET VALUE

# Initialize the result arrays; 
tp_array = np.zeros((len(n_samples),10,n_animals),'int32')
fp_array = np.zeros((len(n_samples),10,n_animals),'int32')
fn_array = np.zeros((len(n_samples),10,n_animals),'int32')

#
for n,n_sample in enumerate(n_samples):
    
    # 
    for k in trange(10):
        
        # ground truth labels
        labels_gt = sleap.load_file(root_dir+"/test_"+str(k)+"_subsample_"+str(n_sample)+".pkg.slp")
        
        # predicted labels
        labels_pr = sleap.load_file(root_dir+"/test_"+str(k)+"_subsample_"+str(n_sample)+".pkg.slp.predictions.slp")

        # get true positive, false positive and false negative results
        tp, fp, fn = get_matrices(cohort, labels_gt, labels_pr)

        # 
        for a in range(n_animals):
            tp_array[n,k,a] = tp[tracks[a]]
            fp_array[n,k,a] = fp[tracks[a]]
            fn_array[n,k,a] = fn[tracks[a]]

        if n_sample==n_samples[-1]:
            break
            
print (" DONE ")

  0%|          | 0/10 [00:00<?, ?it/s]

SLEAP: 1.1.1
TensorFlow: 2.3.1
Numpy: 1.18.5
Python: 3.7.0
OS: Linux-5.4.0-84-generic-x86_64-with-debian-buster-sid


  0%|          | 0/10 [00:00<?, ?it/s]

 DONE 


In [141]:
#####################################
#####################################
#####################################

def plot_scatter(temp, clr, label):
    mean = np.mean(temp)/50.
    std = np.std(temp)/50.

    plt.scatter(n_samples[n], mean, label=label, c=clr)
    plt.errorbar(n_samples[n], mean, yerr=std,
                c=clr)#, xlolims=True, label='xlolims=True')
    return mean
        
        
number_samples = np.arange(5)

if cohort=='cohort1':
    n_animals=4
else:
    n_animals=6

labels = ['true pos','false pos','false neg']
for n in number_samples:

    #
    for a in range(n_animals):
        ax=plt.subplot(2,n_animals//2,a+1)

        temp = tp_array[n,:,a]
        if n==4:
            temp = temp[0]

        clr='black'
        mean = plot_scatter(temp,clr, labels[0])
        
        # false positive
        temp2 = fp_array[n,:,a]
        if n==4:
            temp2 = temp2[0]
        clr='red'
        mean = plot_scatter(temp2,clr, labels[1])

        
        # false negative
        temp2 = fn_array[n,:,a]
        if n==4:
            temp2 = temp2[0]
        clr='blue'
        mean = plot_scatter(temp2,clr, labels[2])
        
        # 
        plt.ylim(0,1.0)
        plt.xlim(0,1000)
        plt.title(tracks[a])
    # #
        if n==0 and a==0:
            plt.legend()
plt.suptitle(cohort+ " "+time_of_day)
plt.show()
        


